### This is a notebook to run the preprocessing pipeline prior to uploading your TIFs to deepcell

In [1]:
# import required packages
import os
import numpy as np
import skimage.io as io
import xarray as xr

from segmentation.utils import data_utils

## This script is currently configured as a template to run with the provided example data. If running your own data, make a copy of this notebook first before modifying it.                         Go to file-> make a copy to create a copy of this notebook

### <span style="color:#ff0000"> All data, images, files, etc. must be placed in the 'data' directory, and referenced via '../data/path_to_your_data' regardless of if it's input or output. </span>

In [ ]:
# load TIFs from GUI-based directory structure
data_dir = "../data/example_dataset/input_data/"

# either get all points in the folder
points = os.listdir(data_dir)
points = [point for point in points if os.path.isdir(data_dir + point) and point.startswith("Point")]

# optionally, select a specific set of points manually
# points = ["Point1", "Point2"]

In [ ]:
# optionally, specify a set of channels to be summed together for better contrast

sum_channels_xr = data_utils.load_imgs_from_tree(data_dir, img_sub_folder="TIFs", fovs=points,
                                                             imgs=["CD3.tif", "CD8.tif"])

channel_sum = np.sum(sum_channels_xr.values, axis=3, dtype="uint8")

new_channel_name = "summed_channel"

summed_xr = xr.DataArray(np.expand_dims(channel_sum, axis=-1), 
                         coords=[sum_channels_xr.fovs, sum_channels_xr.rows,
                                 sum_channels_xr.cols, [new_channel_name]],
                         dims=sum_channels_xr.dims)

tif_saves = [new_channel_name]
for point in summed_xr.fovs.values:
    for tif in tif_saves:
        save_path = os.path.join(data_dir, point, "TIFs", tif + ".tif")
        io.imsave(save_path, summed_xr.loc[point, :, :, tif].values.astype("uint8"))

In [ ]:
# load channels to be included in deepcell data
tifs = ["HH3.tif", "Membrane.tif"]
data_xr = data_utils.load_imgs_from_tree(data_dir, img_sub_folder="TIFs", fovs=points, imgs=tifs)

# save xarray for running through deepcell
data_xr.to_netcdf(os.path.join(data_dir, "deepcell_input.xr"), format="NETCDF3_64BIT")